In [19]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive inside a session. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# with Session(backend=service.backend("ibmq_qasm_simulator")):
#     result = Sampler().run(circuits).result()

In [20]:
import numpy as np
from scipy.stats import norm
import scipy
import json
import csv
from datetime import datetime as dt

In [21]:
cycle_data = [
  {
    "peak": "",
    "trough": "1854-12-01"
  },
  {
    "peak": "1857-06-01",
    "trough": "1858-12-01"
  },
  {
    "peak": "1860-10-01",
    "trough": "1861-06-01"
  },
  {
    "peak": "1865-04-01",
    "trough": "1867-12-01"
  },
  {
    "peak": "1869-06-01",
    "trough": "1870-12-01"
  },
  {
    "peak": "1873-10-01",
    "trough": "1879-03-01"
  },
  {
    "peak": "1882-03-01",
    "trough": "1885-05-01"
  },
  {
    "peak": "1887-03-01",
    "trough": "1888-04-01"
  },
  {
    "peak": "1890-07-01",
    "trough": "1891-05-01"
  },
  {
    "peak": "1893-01-01",
    "trough": "1894-06-01"
  },
  {
    "peak": "1895-12-01",
    "trough": "1897-06-01"
  },
  {
    "peak": "1899-06-01",
    "trough": "1900-12-01"
  },
  {
    "peak": "1902-09-01",
    "trough": "1904-08-01"
  },
  {
    "peak": "1907-05-01",
    "trough": "1908-06-01"
  },
  {
    "peak": "1910-01-01",
    "trough": "1912-01-01"
  },
  {
    "peak": "1913-01-01",
    "trough": "1914-12-01"
  },
  {
    "peak": "1918-08-01",
    "trough": "1919-03-01"
  },
  {
    "peak": "1920-01-01",
    "trough": "1921-07-01"
  },
  {
    "peak": "1923-05-01",
    "trough": "1924-07-01"
  },
  {
    "peak": "1926-10-01",
    "trough": "1927-11-01"
  },
  {
    "peak": "1929-08-01",
    "trough": "1933-03-01"
  },
  {
    "peak": "1937-05-01",
    "trough": "1938-06-01"
  },
  {
    "peak": "1945-02-01",
    "trough": "1945-10-01"
  },
  {
    "peak": "1948-11-01",
    "trough": "1949-10-01"
  },
  {
    "peak": "1953-07-01",
    "trough": "1954-05-01"
  },
  {
    "peak": "1957-08-01",
    "trough": "1958-04-01"
  },
  {
    "peak": "1960-04-01",
    "trough": "1961-02-01"
  },
  {
    "peak": "1969-12-01",
    "trough": "1970-11-01"
  },
  {
    "peak": "1973-11-01",
    "trough": "1975-03-01"
  },
  {
    "peak": "1980-01-01",
    "trough": "1980-07-01"
  },
  {
    "peak": "1981-07-01",
    "trough": "1982-11-01"
  },
  {
    "peak": "1990-07-01",
    "trough": "1991-03-01"
  },
  {
    "peak": "2001-03-01",
    "trough": "2001-11-01"
  },
  {
    "peak": "2007-12-01",
    "trough": "2009-06-01"
  },
  {
    "peak": "2020-02-01",
    "trough": "2020-04-01"
  }
]

In [22]:
'''Takes bond yield data, and only hangs onto the first date recorded in the month'''

with open('AAA10Y.csv', 'r') as AAA:
    #months = {key : [] for key in range(1,13)}
    AAA_data = []
    for line in csv.DictReader(AAA):
        if line['AAA10Y'] == ".":
            continue
        #date = list(line.values())[0].split('-')
        
        #if int(date[0]) in months[int(date[1])]:
        #        continue
                
        #months[int(date[1])].append(int(date[0]))
        AAA_data.append(line)
       
        
print(AAA_data[0])
#print(months)

{'DATE': '1983-01-03', 'AAA10Y': '1.45'}


In [23]:
with open('BAA10Y.csv', 'r') as BAA:
    #months = {key : [] for key in range(1,13)}
    BAA_data = []
    for line in csv.DictReader(BAA):
        if line['BAA10Y'] == ".":
            continue
        #date = list(line.values())[0].split('-')  
        
        #if int(date[0]) in months[int(date[1])]:
        #        continue
                
        #months[int(date[1])].append(int(date[0]))
        
        BAA_data.append(line)

In [24]:
#start = dt.fromisoformat('1950-01-01')
#start = dt.strptime(cycle_data[0]["trough"], "%Y-%m-%d")
start = dt.strptime(BAA_data[0]['DATE'], "%Y-%m-%d")
end = dt.strptime(BAA_data[-1]['DATE'], "%Y-%m-%d")
total_months = round((end - start).days/30.437)
bad_months = 0
print(total_months)
GB = 0
BG = 0

for recession in cycle_data:
    if recession["peak"] == '':
        #trough = dt.strptime(recession["trough"], "%Y-%m-%d")
        continue
        
    peak = dt.strptime(recession["peak"], "%Y-%m-%d")
    
    if peak < start:
        continue
    
    GB += 1
    trough = dt.strptime(recession["trough"], "%Y-%m-%d")
    print(round((trough-peak).days/30.437))
    BG += 1
    bad_months += round((trough-peak).days/30.437)

good_months = total_months - bad_months

pgb = GB/good_months
pbg = BG/bad_months

print(pgb, pbg)

448
8
8
18
2
0.009708737864077669 0.1111111111111111


In [25]:
print(start)
print(end)

1986-01-02 00:00:00
2023-05-03 00:00:00


In [26]:
#start = dt.fromisoformat('1950-01-01')
start = dt.strptime(BAA_data[0]['DATE'], "%Y-%m-%d")

AAA_delta_data = {0 : [], 1: []}


for i, measurement in enumerate(AAA_data):
    
    if measurement['AAA10Y'] == ".":
        continue
    if dt.strptime(measurement['DATE'], "%Y-%m-%d") < start:
        continue
        
    rate = 1 + float(measurement['AAA10Y'])
    #rate = float(measurement['AAA10Y'])
    #rate /= 100
    #rate += 1
    
    
    date = dt.strptime(measurement['DATE'], "%Y-%m-%d")
    
    for recession in cycle_data:
        if recession["peak"] == '':
            continue
        
        peak = dt.strptime(recession["peak"], "%Y-%m-%d")
        
        if peak < start:
            continue
            
        if date < peak:
            AAA_delta_data[0].append(rate)
            break
            
        trough = dt.strptime(recession["trough"], "%Y-%m-%d")
            
        if date < trough:
            AAA_delta_data[1].append(rate)
            break

print(len(AAA_delta_data[0]),len(AAA_delta_data[1]))

7818 747


In [27]:
start = dt.strptime(BAA_data[0]['DATE'], "%Y-%m-%d")

BAA_delta_data = {0 : [], 1: []}


for i, measurement in enumerate(BAA_data):
    
    if measurement['BAA10Y'] == ".":
        continue
    
    rate = 1 + float(measurement['BAA10Y'])
    #rate = float(measurement['BAA10Y'])
    #rate /= 100
    #rate += 1
    
    date = dt.strptime(measurement['DATE'], "%Y-%m-%d")
    
    for recession in cycle_data:
        if recession["peak"] == '':
            continue
        
        peak = dt.strptime(recession["peak"], "%Y-%m-%d")
        
        if peak < start:
            continue
            
        if date < peak:
            BAA_delta_data[0].append(rate)
            break
            
        trough = dt.strptime(recession["trough"], "%Y-%m-%d")
            
        if date < trough:
            BAA_delta_data[1].append(rate)
            break

print(len(BAA_delta_data[0]),len(BAA_delta_data[1]))

7818 747


In [28]:
me_a_good = np.mean(AAA_delta_data[0])
sd_a_good = np.std(AAA_delta_data[0])
md_a_good = np.median(AAA_delta_data[0])

me_b_good = np.mean(BAA_delta_data[0])
sd_b_good = np.std(BAA_delta_data[0])
md_b_good = np.median(BAA_delta_data[0])

corr_array_good = np.corrcoef(AAA_delta_data[0],BAA_delta_data[0])

me_a_bad = np.mean(AAA_delta_data[1])
sd_a_bad = np.std(AAA_delta_data[1])
md_a_bad = np.median(AAA_delta_data[1])

me_b_bad = np.mean(BAA_delta_data[1])
sd_b_bad = np.std(BAA_delta_data[1])
md_b_bad = np.median(BAA_delta_data[1])

corr_array_bad = np.corrcoef(AAA_delta_data[1],BAA_delta_data[1])

print(corr_array_bad)

[[1.         0.84844444]
 [0.84844444 1.        ]]


In [29]:
print(sd_b_bad)

1.2425163496599148


In [30]:
Sigma_good = np.corrcoef(AAA_delta_data[0],BAA_delta_data[0])
Sigma_good[:,0] *= sd_a_good
Sigma_good[0,:] *= sd_a_good
Sigma_good[:,1] *= sd_b_good
Sigma_good[1,:] *= sd_b_good

Sigma_bad = np.corrcoef(AAA_delta_data[1],BAA_delta_data[1])
Sigma_bad[:,0] *= sd_a_bad
Sigma_bad[0,:] *= sd_a_bad
Sigma_bad[:,1] *= sd_b_bad
Sigma_bad[1,:] *= sd_b_bad

mu_good = np.array([[me_a_good],[me_b_good]])
mu_bad = np.array([[me_a_bad],[me_b_bad]])


print(Sigma_good)
print(Sigma_bad)
print(mu_good)
print(mu_bad)

[[0.18018464 0.21486784]
 [0.21486784 0.30936353]]
[[0.38344241 0.65279315]
 [0.65279315 1.54384688]]
[[2.31114607]
 [3.23254925]]
[[2.78427041]
 [4.32749665]]


In [31]:
gamma = [1,2,5,10,60]
weights_good = []

Sigma_good_inv = np.linalg.inv(Sigma_good)
One = np.array([[1],[1]])
One_T = np.transpose(One)

Sigma_bad_inv = np.linalg.inv(Sigma_bad)
One = np.array([[1],[1]])
One_T = np.transpose(One)


for g in gamma:
    v = 2*g 

    v -= One_T@Sigma_good_inv@mu_good

    v /= One_T@Sigma_good_inv@One

    w = Sigma_good_inv@(mu_good+(v*One))
    w /= np.sum(w)
    #print(w)
    growth = np.transpose(w)@mu_bad
    #print('gamma = ',g)
    #print(np.log2(growth[0][0]))
    #print(growth[0][0])
    print(growth)
    
    var = np.transpose(w)@Sigma_good@w
    #print(np.sqrt(var))

[[13.77599927]]
[[7.83270312]]
[[4.26672542]]
[[3.07806619]]
[[2.08751683]]


In [32]:
gamma = [1,2,5,10,60]
weights_bad = []

Sigma_bad_inv = np.linalg.inv(Sigma_bad)
One = np.array([[1],[1]])
One_T = np.transpose(One)

for g in gamma:
    v = 2*g 

    v -= One_T@Sigma_bad_inv@mu_bad

    v /= One_T@Sigma_bad_inv@One
    
    w = Sigma_bad_inv@(mu_bad+(v*One))
    w /= np.sum(w)
    #print(w)
    growth = np.transpose(w)@mu_bad
    print(growth)
    #print(growth[0][0])
    var = np.transpose(w)@Sigma_bad@w
    #print(np.sqrt(var))

[[4.03101444]]
[[3.07334361]]
[[2.49874111]]
[[2.30720695]]
[[2.14759514]]


In [33]:
def ClassicalMC(steps, pg, pb):
    probs = []
    n = steps
    for i in range(2**(n+1)):
        prob = 1
        state = list(map(float,format(i, '0'+str(n+1)+'b')))
        
        if state[0] == 0:
            prob *= (pb)/(pg+pb)
        if state[0] == 1:
            prob *= 1-(pb)/(pg+pb)
        for j in range(n):
            if state[j]==0 and state[j+1]==0:
                prob *= 1-pg
            if state[j]==0 and state[j+1]==1:
                prob *= pg
            if state[j]==1 and state[j+1]==0:
                prob *= pb
            if state[j]==1 and state[j+1]==1:
                prob *= 1-pb
        probs.append(prob)
    return {i:probs[i] for i in range(2**(n+1))}

In [34]:
def draw_random_normal_int(low:int, high:int):

    # generate a random normal number (float)
    normal = np.random.normal(loc=0, scale=1, size=1)

    # clip to -3, 3 (where the bell with mean 0 and std 1 is very close to zero
    normal = -2 if normal < -2 else normal
    normal = 2 if normal > 2 else normal

    # scale range of 6 (-3..3) to range of low-high
    scaling_factor = (high-low) / 4
    normal_scaled = normal * scaling_factor

    # center around mean of range of low high
    normal_scaled += low + (high-low)/2

    # then round and return
    return np.round(normal_scaled)

In [35]:
def NormalBinned(z_size):
    bins = np.zeros(int(2**z_size))
    N = 100000

    for i in range(N):
        index = int(draw_random_normal_int(0,int(2**z_size)-1))
        bins[index] += 1


    bins /= N
    return bins   

In [36]:
def DefaulProb(regime, variable, group):
    if regime == 0:
        default_probs = [0.1,0.2]
        sensitivities = [0.1,0.05]
    if regime == 1:
        default_probs = [0.15,0.25]
        sensitivities = [0.15,0.1]
        
    else:
        default_probs = [0.15,0.25]
        sensitivities = [0.1,0.05]
    
    return norm.cdf((default_probs[group] - (np.sqrt(sensitivities[group])*variable))/(np.sqrt(1-sensitivities[group])))

In [37]:
def ClassicalCreditRisk():
    
    z_size = 3
    groups = [1,2]
    
    prob_0 = 0
    prob_1 = 0
    prob_2 = 0
    prob_3 = 0
    
    Norm = NormalBinned(z_size)
    
    
    for z in range(8):
        prob_outcome = Norm[z]

        prob_default_group = []

        for group in range(len(groups)):
            prob_default_group.append(DefaulProb(None, z, group))

        prob_0 += prob_outcome*(1-prob_default_group[0])*(1-prob_default_group[1])
        prob_1 += prob_outcome*prob_default_group[0]*(1-prob_default_group[1])
        prob_2 += prob_outcome*(1-prob_default_group[0])*prob_default_group[1]
        prob_3 += prob_outcome*prob_default_group[0]*prob_default_group[1]
    
    return [prob_0, prob_0+prob_1, prob_0+prob_1+prob_2, prob_0+prob_1+prob_2+prob_3]
                           
#ClassicalLossDistribution() 

In [38]:
ClassicalCreditRisk()

C:\Users\19899\AppData\Local\Temp\ipykernel_15260\376523967.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  index = int(draw_random_normal_int(0,int(2**z_size)-1))


[np.float64(0.5792635379418261),
 np.float64(0.694081941149454),
 np.float64(0.9233873936538466),
 np.float64(1.0000000000000002)]

In [39]:
def ClassicalMCCreditRisk(pg, pb):
    
    z_size = 3
    groups = [1,2]
    steps = 6
    
    prob_0 = 0
    prob_1 = 0
    prob_2 = 0
    prob_3 = 0
    
    MC = ClassicalMC(steps, pg, pb)
    Norm = NormalBinned(z_size)
    
    for chain in MC.items():
        prob_mc = chain[1]
        state_mc = list(map(float,format(chain[0], '0'+str(steps+1)+'b')))
        for z in range(8):
            prob_norm = Norm[z]
            
            prob_outcome = prob_mc*prob_norm
            
            prob_default_group = []
            
            for group in range(len(groups)):
                prob_default_group.append(DefaulProb(chain[0], z, group))
            
            prob_0 += prob_outcome*(1-prob_default_group[0])*(1-prob_default_group[1])
            prob_1 += prob_outcome*prob_default_group[0]*(1-prob_default_group[1])
            prob_2 += prob_outcome*(1-prob_default_group[0])*prob_default_group[1]
            prob_3 += prob_outcome*prob_default_group[0]*prob_default_group[1]
    
    return [prob_0, prob_0+prob_1, prob_0+prob_1+prob_2, prob_0+prob_1+prob_2+prob_3]
                           
#ClassicalMCLossDistribution(0.3,0.4) 

In [40]:
ClassicalMCCreditRisk(0.009708737864077669, 0.1111111111111111)

C:\Users\19899\AppData\Local\Temp\ipykernel_15260\376523967.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  index = int(draw_random_normal_int(0,int(2**z_size)-1))


[np.float64(0.5808289654222186),
 np.float64(0.6952334605366637),
 np.float64(0.9239770582333439),
 np.float64(0.9999999999999978)]

In [41]:
def ClassicalDerivPriceToMeasurement(price, strike):
    
    integer_precision = 2
    fractional_precision = 6
    c = 0.05
    
    f_max = ((2**(integer_precision+1))-(2.0**(fractional_precision)))
    po_max = f_max - strike
    
    price /= po_max
    price /=  2 / np.pi / c

    measurement = price + 1/2 - np.pi / 4 * c
    
    return measurement

ClassicalDerivPriceToMeasurement(0.15683, 0.90)

0.46051361732403684

In [42]:
def ClassicalDerivPrice():
    steps = 6
    T = 1/12
    
    dt = T/steps
    
    measurements = []
    
    strikes = [0.9,0.95,1.0,1.05,1.10]
    
    for strike in strikes:
        values = []
        
        for i in range(2**(steps)):      
            S = 1.0 
            state_bt = list(map(float,format(i, '0'+str(steps+1)+'b')))

            for j, k in enumerate(state_bt):

                sigma = 0.2+min(max(1.2*i*dt,0),0.1)
                mu = 0.1 - ((sigma**2)/2)

                if k == 0.0:
                    S *= np.e**((mu*dt)+(sigma*np.sqrt(dt)))

                if k == 1.0:
                    S *= np.e**((mu*dt)-(sigma*np.sqrt(dt)))

            if S > strike:
                values.append(S-strike)


        expectation = 0

        for index in range(len(values)):
            v = values[index]
            expectation += v/(2**steps)
            
        measurement_prob = ClassicalDerivPriceToMeasurement(expectation, strike)    
        measurements.append(measurement_prob)
        
    return measurements
    
ClassicalDerivPrice()   

[np.float64(0.46053335797941125),
 np.float64(0.4605954851809811),
 np.float64(0.460647415527079),
 np.float64(0.4606890217931239),
 np.float64(0.4607126931700151)]

In [43]:
def ClassicalMCDerivPrice(pg, pb):
    steps = 3
    T = 1/12
    MC = ClassicalMC(steps, pg, pb)
    
    dt = T/steps
            
    sigma_off = [0.2,0.3]
    r = [0.2,0.1]
    
    measurements = []
    
    strikes = [0.9,0.95,1.0,1.05,1.10]
    
    for strike in strikes:
        values = []
        probs = []
        
        for chain in MC.items():
            prob = chain[1]/(2**steps)
            state_mc = list(map(float,format(chain[0], '0'+str(steps+1)+'b')))
            for i in range(2**(steps)):      
                S = 1.0 
                state_bt = list(map(float,format(i, '0'+str(steps+1)+'b')))

                for j, k in enumerate(state_bt):

                    sigma = sigma_off[int(state_mc[j])]+min(max(1.2*i*dt,0),0.1)
                    mu = r[int(state_mc[j])] - ((sigma**2)/2)

                    if k == 0.0:
                        S *= np.e**((mu*dt)+(sigma*np.sqrt(dt)))

                    if k == 1.0:
                        S *= np.e**((mu*dt)-(sigma*np.sqrt(dt)))

                if S > strike:
                    values.append(S-strike)
                    probs.append(prob)


        expectation = 0

        for index in range(len(values)):
            p = probs[index]
            v = values[index]
            expectation += p*v
            
        measurement_prob = ClassicalDerivPriceToMeasurement(expectation, strike)
        
        measurements.append(measurement_prob)
        
    return measurements
    
ClassicalMCDerivPrice(0.009708737864077669, 0.1111111111111111)   

[np.float64(0.46050882401974397),
 np.float64(0.46057259021969277),
 np.float64(0.4606329122003678),
 np.float64(0.4606758568034812),
 np.float64(0.4607102510253648)]

In [44]:
def CorporateBond(value, steps, growths):
    
    pgb = 0.009708737864077669 
    pbg = 0.1111111111111111
    
    MC = ClassicalMC(steps,pgb,pbg)
    probability = 0
    for path in MC.keys():
        state = list(map(float,format(path, '0'+str(steps+1)+'b')))
        #print(state)
        tot_growth = 0
        for step in range(1,steps+1):
            regime = state[step]
            tot_growth += growths[int(regime)]
        #print(tot_growth)
        #print(MC[path])
        if tot_growth <= value:
            probability += MC[path]
    return probability

In [45]:
CorporateBond(0, 3, [0.771, 0])

0.06349206349206343

In [46]:
MontC = ClassicalMC(3,0.001,0.3)

In [47]:
def SingleMSE(classical, quantum):
    
    MSE = 0
    
    for i, quant in enumerate(quantum.values()):
        error = quant - classical[i]
        MSE += error**2
    
    return MSE/len(quantum)

In [50]:
def QuantDictID(keys, jobid):
    '''
    job = service.runtime.job(jobid)
    result = job.result().quasi_dists
    resiliences = [0,3]
    '''
    resiliences = [0]
    result = [{1: 0.57}, {1: 0.69}, {1:  0.93}, {1: 1.0 },
             {1: 0.66}, {1: 0.78 }, {1: 0.96}, {1: 1.0},
              {1:  0.64}, {1:  0.75}, {1:  0.95}, {1:  1.0}
             ]
    
    groups = round(len(result)/(len(resiliences)*(len(keys))*3))
    
    qdict = {res: {param : {group : [] for group in range(groups)} for param in range(3)} for res in resiliences}
    
    for i, res in enumerate(resiliences):
        
        for params in range(3):
        
            for group in range(groups):

                for j, key in enumerate(keys):
                    index = (i*groups*3*len(keys)*groups)+(params*len(keys)*groups)+(j*groups)+group
                    qdict[res][params][group].append(result[index][1])
        
    return [groups, qdict]
    
#QuantDictID([0,1,2,3],"cgkanstg6pokv26293cg")

In [51]:
def ClassDict(keys):
    cdict = {}
    
    if keys == [0.9,0.95,1.0,1.05,1.10]:
        cdict[0] = ClassicalDerivPrice()
        cdict[1] = ClassicalMCDerivPrice(0.009708737864077669, 0.1111111111111111)
        cdict[2] = ClassicalMCDerivPrice(0.3, 0.4)
        
    if keys == [0,1,2,3]:
        cdict[0] = ClassicalCreditRisk()
        cdict[1] = ClassicalMCCreditRisk(0.009708737864077669, 0.1111111111111111)
        cdict[2] = ClassicalMCCreditRisk(0.3, 0.4)
        
    return cdict
        
#print(len(ClassDict([3,6,12,24])))

In [52]:
def MultiMSEID(keys, jobid):
        
    classical_dict = ClassDict(keys)
    quantum_dict = QuantDictID(keys, jobid)
    groups = quantum_dict[0]
    quantum_dict = quantum_dict[1]
    
    resiliences = [0,3]
    MSE_res = {res : {params : "" for params in range(3)} for res in resiliences}
    
    for res in quantum_dict.keys():
        for params in range(3):
            classical = classical_dict[params]
            quantum = quantum_dict[res][params]
            error = 0
            for group in quantum.keys():
                error += SingleMSE(classical, quantum[group])
            
        
            MSE_res[res][params] = error/groups
    return MSE_res

#MultiMSEID([0,1,2,3],"cgkalkpnnc3hiktnb390")

In [53]:
def MCQuantDictFile(keys, filename):
    '''
    job = service.runtime.job(jobid)
    result = job.result().quasi_dists
    resiliences = [0,3]
    '''
    
    with open(filename, "r") as file:
        result = json.load(file)['0']
    
    #result = [{1: 0.57}, {1: 0.69}, {1:  0.93}, {1: 1.0 },
    #         {1: 0.66}, {1: 0.78 }, {1: 0.96}, {1: 1.0},
    #          {1:  0.64}, {1:  0.75}, {1:  0.95}, {1:  1.0}
    #         ]
    
    iterations = range(len(result))
    params_options = [0,1,2]
    
    qdict = {params: {key : {iteration : "" for iteration in iterations} for key in keys} for params in params_options}
    
    for iteration in iterations:
        index = 0
            
        for key in keys:

            for params in params_options:

                quasi_dists = result[iteration][index]
                qdict[params][key][iteration] = quasi_dists
                index +=1
        
    return qdict
    
#test_dict = MCQuantDictFile([3,6,12,25],"MC_Real.json")
#print(test_dict[0][3][1])

In [54]:
def MC_MSE_File(steps_options, filename):
        
    quantum_dict = MCQuantDictFile(steps_options, filename)
    
    MSE_dict = {params : {steps : "" for steps in steps_options} for params in range(3)}
    
    pgb_options = [0.3, 0.02418207681365576, 0.009708737864077669]
    pbg_options = [0.4, 0.058823529411764705, 0.1111111111111111]
    
    for params in range(3):
        
        for steps in steps_options:

            classical = ClassicalMC(steps, pgb_options[params], pbg_options[params])
            quantum = quantum_dict[params][steps]
            error = 0
            iterations = len(quantum)
            
            for iteration in quantum.keys():
                error += SingleMSE(classical, quantum[iteration])


            MSE_dict[params][steps] = error/iterations
            
    return MSE_dict

In [ ]:
MC_MSE_File([3,6,12,24], "MC_Fake2.json")

{0: {3: 1.332673738440688e-05,
  6: 3.7890488605193183e-06,
  12: 1.8827932857387863e-07,
  24: 6.175451605584057e-08},
 1: {3: 0.004149948475495823,
  6: 0.0010012610264926638,
  12: 0.00014352849537831603,
  24: 1.204210495828633e-05},
 2: {3: 0.0002759959408972652,
  6: 5.7543554078051494e-05,
  12: 9.580034917656184e-06,
  24: 1.9919750743293185e-06}}

In [ ]:
def QuantDictFile(keys, filename):
    '''
    job = service.runtime.job(jobid)
    result = job.result().quasi_dists
    resiliences = [0,3]
    '''
    
    with open(filename, "r") as file:
        result = json.load(file)['0']
    
    #result = [{1: 0.57}, {1: 0.69}, {1:  0.93}, {1: 1.0 },
    #         {1: 0.66}, {1: 0.78 }, {1: 0.96}, {1: 1.0},
    #          {1:  0.64}, {1:  0.75}, {1:  0.95}, {1:  1.0}
    #         ]
    
    iterations = range(len(result)-64)
    params_options = [0,1,2]
    
    qdict = {params: {iteration : {key: "" for key in keys} for iteration in iterations} for params in params_options}
    
    for iteration in iterations:
        index = 0
            
        for key in keys:

            for params in params_options:
                
                quasi_dists = result[iteration+64][index]
                qdict[params][iteration][key] = quasi_dists
                index +=1
        
    return qdict

In [ ]:
def MSE_file(keys, filename):
    quantum_dict=QuantDictFile(keys, filename)
    iterations = len(quantum_dict[0])
    
    classical_dict = ClassDict(keys)
    MSE_dict = {params : "" for params in range(3)}
    
    for params in range(3):
        
        classical = classical_dict[params]
        
        error = 0
        
        for it in range(iterations):
            
            quantum = {key : quantum_dict[params][it][key]['1'] for key in keys}
            error += SingleMSE(classical, quantum)
            
        MSE_dict[params] = error/iterations
    
    return MSE_dict
            

In [ ]:
MSE_file([0,1,2,3], "CreditRisk_Fake2.json")

{0: 0.002061988681704327, 1: 0.00017903420423567975, 2: 0.0002003679040123542}